<a href="https://colab.research.google.com/github/prasannaml/Gurobi-in-Python-/blob/main/Supply_network_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Supply Network optimization example

## Objective and Prerequisites

Given a set of dairy farms,storage facilities and stores we will use mathematical optimization to determine the best way to satisfy demand while minimizing shipping costs.
  



# Problem Description

We have six stores who are end customers for the dairy farm company. Demand can be satisfied from the 4 depots or directly from the two farms.Each depot can support a maximum volume of dairy through it and each
farm can produce a maximum amount of dairy in a day.There are known costs associated with transporting the dairy, from a farm to a depot, from a depot to a store, or from a farm directly to a store.

Two farms are in Lansdale PA and West Haven  CT

| Farm | Supply (gallons) |
| --- | --- |
| Lansdale | 150,000 |
| West Haven |  200,000 |

The dairy can be shipped from a factory to a set of four depots. Each depot has a maximum throughput. Depots don't produce or consume the product; they simply pass the product on to customers.



| Depot | Throughput (gallons) |
| --- | --- |
| Philadelphia | 70,000 |
| Pittsburgh | 50,000 |
| New York  | 100,000 |
| Boston | 40,000 |




Our network has six stores, each with a given demand.

| Store | Demand (gallons) |
| --- | --- |
| S1 | 50,000 |
| S2 | 10,000 |
| S3 | 40,000 |
| S4 | 35,000 |
| S5 | 60,000 |
| S6 | 20,000 |


Shipping costs are given in the following table (in dollars per gallon). Columns are source cities and rows are destination cities. Thus, for example, it costs $1 per gallon to ship the product from Lansdale to NYC. A '-' in the table indicates that that combination is not possible, so for example it is not possible to ship from the factory in West Haven to the depot in Philadelphia.


| To | Lansdale | WestHaven | Philadelphia | Pittsburgh | New York | Boston |
| --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Philadelphia  | 0.5 |   - |
| Pittsburgh | 0.5 | 0.3 |
| New York     | 1.0 | 0.5 |
| Boston     | 0.2 | 0.2 |
| Customers |
| S1 | 1.0 | 2.0 |   - | 1.0 |   - |   - |
| S2 |   - |   - | 1.5 | 0.5 | 1.5 |   - |
| S3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 |
| S4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 |
| S5 |   - |   - |   - | 0.5 | 0.5 | 0.5 |
| S6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 |


The question to be answered is how to satisfy the demands of the end customers while minimizing shipping costs.

## Model Formulation

### Sets and Indices

$f \in \text{Farms}=\{\text{Lansdale}, \text{West Haven}\}$

$d \in \text{Depots}=\{\text{Philadelphia}, \text{Pittsburgh}, \text{New York}, \text{Boston}\}$

$c \in \text{Stores}=\{\text{S1}, \text{S2}, \text{S3}, \text{S4}, \text{S5}, \text{S6}\}$

$\text{Cities} = \text{Farms} \cup \text{Depots} \cup \text{Stores}$

### Parameters

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one gallon from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from farm $f$ (in gallons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in gallons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at stores $c$ (in gallons).

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in gallons) that is shipped from source $s$ to destionation $t$.


### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}

### Constraints

- **Farm output**: Flow of goods from a farm must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} =
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity**: Flow into a depot must respect depot capacity.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{Depots}
\end{equation}

In [1]:
%pip install gurobipy

import pandas as pd
import gurobipy  as gp
from gurobipy import GRB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 75.8 MB/s eta 0:00:00


In [ ]:
## INPUT DATA

In [2]:
#dictionaries to store supply,depot throughput and store demand

supply = dict({"Lansdale": 150000,
              "West Haven": 200000})
through = dict({"Philadelphia":70000,
                "Pittsburgh":50000,
                "New York":100000,
                "Boston":40000})

demand= dict({ 'S1': 50000,
               'S2': 10000,
               'S3': 40000,
               'S4': 35000,
               'S5': 60000,
               'S6': 20000})


arcs,cost = gp.multidict({
    ('Lansdale', 'Philadelphia'): 0.5,
    ('Lansdale', 'Pittsburgh'): 0.5,
    ('Lansdale', 'New York'): 1.0,
    ('Lansdale', 'Boston'): 0.2,
    ('Lansdale', 'S1'): 1.0,
    ('Lansdale', 'S3'): 1.5,
    ('Lansdale', 'S4'): 2.0,
    ('Lansdale', 'S6'): 1.0,
    ('West Haven', 'Pittsburgh'): 0.3,
    ('West Haven', 'New York'): 0.5,
    ('West Haven', 'Boston'): 0.2,
    ('West Haven', 'S1'): 2.0,
    ('Philadelphia', 'S2'): 1.5,
    ('Philadelphia', 'S3'): 0.5,
    ('Philadelphia', 'S5'): 1.5,
    ('Philadelphia', 'S6'): 1.0,
    ('Pittsburgh', 'S1'): 1.0,
    ('Pittsburgh', 'S2'): 0.5,
    ('Pittsburgh', 'S3'): 0.5,
    ('Pittsburgh', 'S4'): 1.0,
    ('Pittsburgh', 'S5'): 0.5,
    ('New York', 'S2'): 1.5,
    ('New York', 'S3'): 2.0,
    ('New York', 'S5'): 0.5,
    ('New York', 'S6'): 1.5,
    ('Boston', 'S3'): 0.2,
    ('Boston', 'S4'): 1.5,
    ('Boston', 'S5'): 0.5,
    ('Boston', 'S6'): 1.5


})

In [ ]:
## Model Deployment

In [4]:
model =gp.Model('SupplyNetworkDesign')
ship = model.addVars(arcs,obj=cost,name="ship") # adding variables for decision variables and objective is optimizing cost variable

Restricted license - for non-production use only - expires 2024-10-28


In [12]:
ship

{('Lansdale', 'Philadelphia'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'Pittsburgh'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'New York'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'S1'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'S3'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'S4'): <gurobi.Var *Awaiting Model Update*>,
 ('Lansdale', 'S6'): <gurobi.Var *Awaiting Model Update*>,
 ('West Haven', 'Pittsburgh'): <gurobi.Var *Awaiting Model Update*>,
 ('West Haven', 'New York'): <gurobi.Var *Awaiting Model Update*>,
 ('West Haven', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('West Haven', 'S1'): <gurobi.Var *Awaiting Model Update*>,
 ('Philadelphia', 'S2'): <gurobi.Var *Awaiting Model Update*>,
 ('Philadelphia', 'S3'): <gurobi.Var *Awaiting Model Update*>,
 ('Philadelphia', 'S5'): <gurobi.Var *Awaiting Model Update*>,
 ('Philadelphia', 'S6'): <gurobi.Var *Awaiting Mo

In [5]:
# Dairy production constraint

farms= supply.keys()

farms_ship = model.addConstrs((gp.quicksum(ship.select(farm,'*'))<= supply[farm] for farm in farms),name="farm")

In [8]:
# store demand constraint

stores = demand.keys()

stores_ship = model.addConstrs((gp.quicksum(ship.select('*',store))==demand[store] for store in stores), name ="storedemand")


In [9]:
#depot through capacity

depots =through.keys()

depot_capacity = model.addConstrs((gp.quicksum(ship.select('*',depot))<= through[depot] for depot in depots),name="depotthrough" )

In [10]:
#depot in equal to out

depot_in_out = model.addConstrs((gp.quicksum(ship.select(depot,'*'))== gp.quicksum(ship.select('*',depot)) for depot in depots),name="depotequal")

In [13]:
#optimize model


model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 29 columns and 65 nonzeros
Model fingerprint: 0xa543cd23
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 15 rows, 29 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.812500e+04   0.000000e+00      0s
       7    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.985000000e+05


In [21]:
model.objval

198500.0

Our total costs is optimized to be $198500


In [19]:
#shipping order and combinations of solution

ship_order = pd.DataFrame(columns=["From","To","Volume"])
for arc in arcs:
  if ship[arc].x>0:
    ship_order = ship_order.append({"From":arc[0],"To":arc[1],"Volume":ship[arc].x},ignore_index=True)
ship_order.index =[''] * len(ship_order)
ship_order



<ipython-input-19-110297a68e29>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ship_order = ship_order.append({"From":arc[0],"To":arc[1],"Volume":ship[arc].x},ignore_index=True)
<ipython-input-19-110297a68e29>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ship_order = ship_order.append({"From":arc[0],"To":arc[1],"Volume":ship[arc].x},ignore_index=True)
<ipython-input-19-110297a68e29>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ship_order = ship_order.append({"From":arc[0],"To":arc[1],"Volume":ship[arc].x},ignore_index=True)
<ipython-input-19-110297a68e29>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ship_order = ship_order

,From,To,Volume
,Lansdale,S1,50000.0
,Lansdale,S6,20000.0
,West Haven,Pittsburgh,50000.0
,West Haven,New York,55000.0
,West Haven,Boston,40000.0
,Pittsburgh,S2,10000.0
,Pittsburgh,S4,35000.0
,Pittsburgh,S5,5000.0
,New York,S5,55000.0
,Boston,S3,40000.0
